## 3a)

In [1]:
#Import libraries
from pyspark.sql import SparkSession
from pyspark.sql import functions as f
from pyspark.sql.functions import *
from functools import reduce

In [2]:
#Start spark session
spark = SparkSession \
    .builder \
    .appName("ICT330 TMA Q3") \
    .config("spark.some.config.option", "some-value") \
    .getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/10/18 14:00:14 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
23/10/18 14:00:14 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [3]:
#Read csv file and store into dataframe
flights_data = spark\
.read\
.option("inferSchema", "true")\
.option("header", "true")\
.csv ("flights_data_v2.csv")

In [4]:
#Show the type of flights_data to check if its a dataframe
print(type(flights_data))

<class 'pyspark.sql.dataframe.DataFrame'>


In [5]:
#Show the content of flights_data
flights_data.show()

+----+-----+---+--------+---------+--------+---------+-------+-------+------+------+----+--------+--------+----+------+
|year|month|day|dep_time|dep_delay|arr_time|arr_delay|carrier|tailnum|flight|origin|dest|air_time|distance|hour|minute|
+----+-----+---+--------+---------+--------+---------+-------+-------+------+------+----+--------+--------+----+------+
|2014|   12|  8|     658|       -7|     935|       -5|     VX| N846VA|  1780|   SEA| LAX|     132|     954|   6|    58|
|2014|    1| 22|    1040|        5|    1505|        5|     AS| N559AS|   851|   SEA| HNL|     360|    2677|  10|    40|
|2014|    3|  9|    1443|       -2|    1652|        2|     VX| N847VA|   755|   SEA| SFO|     111|     679|  14|    43|
|2014|    4|  9|    1705|       45|    1839|       34|     WN| N360SW|   344|   PDX| SJC|      83|     569|  17|     5|
|2014|    3|  9|     754|       -1|    1015|        1|     AS| N612AS|   522|   SEA| BUR|     127|     937|   7|    54|
|2014|    1| 15|    1037|        7|    1

In [6]:
#Show the number of occurences 
flights_data.count()

10000

In [7]:
#Show the schema
flights_data.printSchema()

root
 |-- year: integer (nullable = true)
 |-- month: integer (nullable = true)
 |-- day: integer (nullable = true)
 |-- dep_time: integer (nullable = true)
 |-- dep_delay: integer (nullable = true)
 |-- arr_time: integer (nullable = true)
 |-- arr_delay: integer (nullable = true)
 |-- carrier: string (nullable = true)
 |-- tailnum: string (nullable = true)
 |-- flight: integer (nullable = true)
 |-- origin: string (nullable = true)
 |-- dest: string (nullable = true)
 |-- air_time: integer (nullable = true)
 |-- distance: integer (nullable = true)
 |-- hour: integer (nullable = true)
 |-- minute: integer (nullable = true)



In [8]:
#Display rows with missing values 
rows_with_missing_data = flights_data.where(reduce(lambda x, y: x | y, (f.col(x).isNull() for x in flights_data.columns)))
rows_with_missing_data.show()

+----+-----+---+--------+---------+--------+---------+-------+-------+------+------+----+--------+--------+----+------+
|year|month|day|dep_time|dep_delay|arr_time|arr_delay|carrier|tailnum|flight|origin|dest|air_time|distance|hour|minute|
+----+-----+---+--------+---------+--------+---------+-------+-------+------+------+----+--------+--------+----+------+
|2014|    4|  6|    1329|        4|    2159|     null|     DL| N130DL|  1929|   SEA| ATL|    null|    2182|  13|    29|
|2014|    3|  4|    null|     null|    null|     null|     UA|   null|   156|   SEA| DEN|    null|    1024|null|  null|
|2014|    2| 12|    null|     null|    null|     null|     AS| N527AS|     2|   SEA| DCA|    null|    2329|null|  null|
|2014|    7|  1|    null|     null|    null|     null|     WN| N8323C|  2485|   SEA| MDW|    null|    1733|null|  null|
|2014|   12| 27|    1420|       40|    2012|     null|     OO| N224AG|  3452|   SEA| HDN|    null|     891|  14|    20|
|2014|    4| 30|    null|     null|    n

In [9]:
#Display the count of missing values
rows_with_missing_data.count()

75

In [10]:
#Drop missing values 
flights_data_new = flights_data.na.drop()

#Show the count of rows in the original and cleaned DataFrames
print("Original DataFrame Count:", flights_data.count())
print("Cleaned DataFrame Count:", flights_data_new.count())

Original DataFrame Count: 10000
Cleaned DataFrame Count: 9925


## 3b)

In [11]:
#Find no. of flights per year,month and rank in descending order
flights_data_new.groupBy("year","month").count().sort(desc("count")).show()

+----+-----+-----+
|year|month|count|
+----+-----+-----+
|2014|    8| 1058|
|2014|    7|  973|
|2014|    6|  883|
|2014|    5|  840|
|2014|    9|  833|
|2014|   10|  811|
|2014|    4|  810|
|2014|   12|  799|
|2014|    3|  787|
|2014|   11|  777|
|2014|    1|  732|
|2014|    2|  622|
+----+-----+-----+



In [12]:
#Find no. of flights per day and rank in descending order
flights_per_day_count = flights_data_new.groupBy("day").count()
flights_data_new.groupBy("day").count().sort(desc("count")).show(flights_per_day_count.count(),False)

+---+-----+
|day|count|
+---+-----+
|1  |358  |
|3  |356  |
|28 |349  |
|17 |349  |
|15 |345  |
|9  |345  |
|2  |345  |
|21 |339  |
|30 |339  |
|10 |337  |
|14 |329  |
|5  |328  |
|6  |326  |
|18 |324  |
|29 |323  |
|22 |322  |
|27 |321  |
|23 |321  |
|7  |320  |
|26 |315  |
|16 |314  |
|24 |313  |
|13 |312  |
|12 |310  |
|19 |309  |
|25 |309  |
|8  |305  |
|20 |301  |
|4  |293  |
|11 |291  |
|31 |177  |
+---+-----+



In [13]:
#Find no. of flights per carrier, compute percentage and rank in descending order
flights_data_new.groupBy("carrier") \
  .count() \
  .withColumn('percentage', (f.col('count') / flights_data_new.count()) * 100 ) \
  .withColumn("percentage", f.round('percentage', 2)) \
  .sort(desc("count")) \
  .show()

+-------+-----+----------+
|carrier|count|percentage|
+-------+-----+----------+
|     AS| 3771|     37.99|
|     WN| 1384|     13.94|
|     OO| 1163|     11.72|
|     DL| 1077|     10.85|
|     UA| 1039|     10.47|
|     AA|  476|       4.8|
|     US|  364|      3.67|
|     B6|  213|      2.15|
|     VX|  186|      1.87|
|     F9|  181|      1.82|
|     HA|   71|      0.72|
+-------+-----+----------+



In [14]:
#Compute no.of flights from origin airports and rank in descending order
flights_data_new.groupBy("origin").count().sort(desc("count")).show()

#Compute no. of flights from destination airports and rank in descending order
flights_count = flights_data_new.groupBy("dest").count()
flights_data_new.groupBy("dest").count().sort(desc("count")).show(flights_count.count(),False)

+------+-----+
|origin|count|
+------+-----+
|   SEA| 6711|
|   PDX| 3214|
+------+-----+

+----+-----+
|dest|count|
+----+-----+
|SFO |779  |
|LAX |611  |
|DEN |580  |
|PHX |526  |
|LAS |520  |
|ANC |449  |
|ORD |431  |
|SLC |396  |
|SJC |369  |
|DFW |369  |
|OAK |333  |
|SMF |283  |
|SAN |269  |
|ATL |256  |
|MSP |238  |
|IAH |223  |
|SNA |197  |
|LGB |174  |
|JFK |170  |
|PDX |154  |
|HNL |142  |
|EWR |142  |
|SEA |138  |
|BUR |137  |
|BOS |128  |
|ONT |128  |
|DTW |123  |
|GEG |105  |
|IAD |92   |
|OGG |89   |
|MDW |82   |
|FAI |80   |
|PHL |80   |
|CLT |78   |
|KTN |76   |
|JNU |76   |
|DCA |68   |
|ABQ |57   |
|RDM |54   |
|KOA |52   |
|TUS |50   |
|PSP |47   |
|FAT |47   |
|SBA |46   |
|MCI |45   |
|EUG |39   |
|LIH |37   |
|BWI |34   |
|AUS |31   |
|MCO |31   |
|FLL |29   |
|COS |26   |
|OMA |25   |
|STL |24   |
|HOU |22   |
|RNO |21   |
|MIA |19   |
|SAT |18   |
|MKE |16   |
|TPA |11   |
|MSY |9    |
|BNA |9    |
|BOI |7    |
|LMT |7    |
|SIT |6    |
|CVG |5    |
|BLI |5    |

In [15]:
#Find top 10 planes that made the most flights
flights_data_new.groupBy("tailnum") \
  .count() \
  .sort(desc("count")) \
  .limit(10) \
  .show()

+-------+-----+
|tailnum|count|
+-------+-----+
| N612AS|   50|
| N219AG|   50|
| N224AG|   49|
| N218AG|   49|
| N223AG|   49|
| N611AS|   46|
| N607AS|   45|
| N225AG|   45|
| N435AS|   45|
| N615AS|   44|
+-------+-----+



## 3c) 

In [16]:
#Compute no. of flight departures per hour and rank in descending order
flights_per_hour = flights_data_new.groupBy("hour").count()
flights_data_new.groupBy("hour").count().sort(desc("count")).show(flights_per_hour.count(),False)

+----+-----+
|hour|count|
+----+-----+
|6   |896  |
|10  |802  |
|11  |721  |
|7   |708  |
|8   |658  |
|13  |649  |
|18  |568  |
|12  |539  |
|14  |538  |
|15  |465  |
|9   |454  |
|5   |430  |
|19  |409  |
|17  |394  |
|16  |388  |
|20  |354  |
|22  |310  |
|21  |281  |
|23  |250  |
|0   |89   |
|1   |17   |
|2   |4    |
|24  |1    |
+----+-----+



In [17]:
#Find average positive delay based on carrier and rank in descending order
average_positive_delay = flights_data_new.groupBy("carrier") \
  .agg(avg(when(col("dep_delay") >= 0, col("dep_delay"))) \
  .alias("average positive delay")) \
  .orderBy("average positive delay", ascending=False) \
  .withColumn("average positive delay", f.round(col("average positive delay"), 2))

average_positive_delay.show()

#Find Average departure delay
average_positive_delay.agg({'average positive delay': 'avg'}).show()

#Find average departure delay per month and rank in descending order
average_month_positive_delay = flights_data_new.groupBy("month") \
  .agg(avg(when(col("dep_delay") >= 0, col("dep_delay"))) \
  .alias("average positive delay")) \
  .orderBy("average positive delay", ascending=False) \
  .withColumn("average positive delay", f.round(col("average positive delay"), 2))

average_month_positive_delay.show()

#Find average departure delay per hour and rank in descending order
average_hour_positive_delay = flights_data_new.groupBy("hour") \
  .agg(avg(when(col("dep_delay") >= 0, col("dep_delay"))) \
  .alias("average positive delay")) \
  .orderBy("average positive delay", ascending=False) \
  .withColumn("average positive delay", f.round(col("average positive delay"), 2))

average_hour_positive_delay.show(average_hour_positive_delay.count(),False)

+-------+----------------------+
|carrier|average positive delay|
+-------+----------------------+
|     AA|                 33.97|
|     VX|                 32.95|
|     HA|                 32.71|
|     OO|                 30.74|
|     B6|                 24.83|
|     UA|                 24.41|
|     F9|                 22.26|
|     DL|                 21.65|
|     WN|                 20.36|
|     US|                 17.85|
|     AS|                 17.27|
+-------+----------------------+

+---------------------------+
|avg(average positive delay)|
+---------------------------+
|         25.363636363636363|
+---------------------------+

+-----+----------------------+
|month|average positive delay|
+-----+----------------------+
|    4|                 27.59|
|    2|                 26.97|
|   12|                 25.57|
|    1|                 24.37|
|    3|                 21.81|
|    9|                  21.2|
|    7|                 20.99|
|    8|                 20.28|
|    5|     

In [18]:
#Find average negative delay based on carrier and rank in descending order
average_negative_delay = flights_data_new.groupBy("carrier") \
  .agg(avg(when(col("dep_delay") < 0, col("dep_delay"))) \
  .alias("average negative delay")) \
  .orderBy("average negative delay", ascending=False) \
  .withColumn("average negative delay", f.round(col("average negative delay"), 2))

average_negative_delay.show()

#Find Average negative departure delay
average_negative_delay.agg({'average negative delay': 'avg'}).show()

#Find average negative departure delay per month and rank in descending order
average_month_negative_delay = flights_data_new.groupBy("month") \
  .agg(avg(when(col("dep_delay") < 0, col("dep_delay"))) \
  .alias("average negative delay")) \
  .orderBy("average negative delay", ascending=False) \
  .withColumn("average negative delay", f.round(col("average negative delay"), 2))

average_month_negative_delay.show()

#Find average negative departure delay per hour and rank in descending order
average_hour_negative_delay = flights_data_new.groupBy("hour") \
  .agg(avg(when(col("dep_delay") < 0, col("dep_delay"))) \
  .alias("average negative delay")) \
  .orderBy("average negative delay", ascending=False) \
  .withColumn("average negative delay", f.round(col("average negative delay"), 2))

average_hour_negative_delay.show(average_hour_negative_delay.count(),False)

+-------+----------------------+
|carrier|average negative delay|
+-------+----------------------+
|     WN|                 -2.87|
|     DL|                 -3.93|
|     VX|                 -4.52|
|     US|                 -4.57|
|     AS|                 -4.76|
|     UA|                 -4.87|
|     AA|                 -5.25|
|     OO|                 -5.74|
|     B6|                 -5.79|
|     HA|                  -6.2|
|     F9|                  -6.3|
+-------+----------------------+

+---------------------------+
|avg(average negative delay)|
+---------------------------+
|        -4.9818181818181815|
+---------------------------+

+-----+----------------------+
|month|average negative delay|
+-----+----------------------+
|    8|                 -4.37|
|    7|                 -4.42|
|   12|                 -4.49|
|    2|                 -4.54|
|    6|                 -4.56|
|    3|                 -4.68|
|    1|                 -4.72|
|    9|                 -4.86|
|   11|     

## 3d)

In [19]:
#Find average,min,max flight distance of carrier and rank in descending order based on average distance
flights_data_new.groupBy("carrier") \
  .agg(
        f.avg(f.col('distance')).alias('avg_distance'),
        f.min(f.col('distance')).alias('min_distance'),
        f.max(f.col('distance')).alias('max_distance'),
    ) \
  .orderBy('avg_distance', ascending=False) \
  .withColumn("avg_distance", f.round(col('avg_distance'), 2)) \
  .show()


+-------+------------+------------+------------+
|carrier|avg_distance|min_distance|max_distance|
+-------+------------+------------+------------+
|     HA|      2630.1|        2603|        2677|
|     AA|     1738.17|        1107|        2724|
|     B6|     1631.27|         846|        2537|
|     DL|     1563.95|         630|        2677|
|     US|     1525.47|        1009|        2406|
|     UA|     1454.97|         550|        2434|
|     AS|     1218.03|          93|        2717|
|     F9|     1033.72|         991|        2021|
|     WN|      906.71|         444|        2358|
|     VX|      759.01|         550|         954|
|     OO|       604.8|         106|        1368|
+-------+------------+------------+------------+



In [20]:
#Create new column "flight_speed(miles per hour)". Unit of flight distance is miles and air time is minutes.
#Speed = distance / Time = distance/(air_time/60)

flights_data_final = flights_data_new.withColumn('flight_speed(miles per hour)', 
                            f.round(flights_data_new.distance / (flights_data_new.air_time / 60),2)) 

flights_data_final.show()

+----+-----+---+--------+---------+--------+---------+-------+-------+------+------+----+--------+--------+----+------+----------------------------+
|year|month|day|dep_time|dep_delay|arr_time|arr_delay|carrier|tailnum|flight|origin|dest|air_time|distance|hour|minute|flight_speed(miles per hour)|
+----+-----+---+--------+---------+--------+---------+-------+-------+------+------+----+--------+--------+----+------+----------------------------+
|2014|   12|  8|     658|       -7|     935|       -5|     VX| N846VA|  1780|   SEA| LAX|     132|     954|   6|    58|                      433.64|
|2014|    1| 22|    1040|        5|    1505|        5|     AS| N559AS|   851|   SEA| HNL|     360|    2677|  10|    40|                      446.17|
|2014|    3|  9|    1443|       -2|    1652|        2|     VX| N847VA|   755|   SEA| SFO|     111|     679|  14|    43|                      367.03|
|2014|    4|  9|    1705|       45|    1839|       34|     WN| N360SW|   344|   PDX| SJC|      83|     569

In [21]:
#Find average,min,max flight speed of carrier and rank in descending order based on average speed
flights_data_final.groupBy("carrier") \
  .agg(
        f.avg(f.col('flight_speed(miles per hour)')).alias('avg_speed'),
        f.min(f.col('flight_speed(miles per hour)')).alias('min_speed'),
        f.max(f.col('flight_speed(miles per hour)')).alias('max_speed'),
    ) \
  .orderBy('avg_speed', ascending=False) \
  .withColumn("avg_speed", f.round(col('avg_speed'), 2)) \
  .show()

+-------+---------+---------+---------+
|carrier|avg_speed|min_speed|max_speed|
+-------+---------+---------+---------+
|     AA|   509.16|   423.83|   579.07|
|     DL|   500.53|   311.66|   600.55|
|     US|   491.35|   380.75|   605.04|
|     F9|   483.45|   407.26|   560.94|
|     B6|   482.02|   357.41|    576.0|
|     UA|   478.66|    275.0|   588.69|
|     HA|   473.33|   415.75|   538.55|
|     AS|   451.77|   199.29|   652.34|
|     WN|   446.95|   322.57|   557.48|
|     VX|   421.25|   331.22|   511.07|
|     OO|   374.65|   157.96|   566.07|
+-------+---------+---------+---------+



In [22]:
#Find shortest flight from "PDX" based on distance
flights_data_final.filter(col("origin") == "PDX").orderBy(col("distance").asc()).limit(1).show()

#Find longest flight from "SEA" based on duration
flights_data_final.filter(col("origin") == "SEA").orderBy(col("air_time").desc()).limit(1).show()


+----+-----+---+--------+---------+--------+---------+-------+-------+------+------+----+--------+--------+----+------+----------------------------+
|year|month|day|dep_time|dep_delay|arr_time|arr_delay|carrier|tailnum|flight|origin|dest|air_time|distance|hour|minute|flight_speed(miles per hour)|
+----+-----+---+--------+---------+--------+---------+-------+-------+------+------+----+--------+--------+----+------+----------------------------+
|2014|    3| 20|    1200|       -8|    1239|       -7|     OO| N569SW|  5411|   PDX| EUG|      30|     106|  12|     0|                       212.0|
+----+-----+---+--------+---------+--------+---------+-------+-------+------+------+----+--------+--------+----+------+----------------------------+

+----+-----+---+--------+---------+--------+---------+-------+-------+------+------+----+--------+--------+----+------+----------------------------+
|year|month|day|dep_time|dep_delay|arr_time|arr_delay|carrier|tailnum|flight|origin|dest|air_time|distanc

In [23]:
#Find average flight duration of carrier “UA” and originated from “SEA”. 
average_duration = flights_data_final.filter(col("carrier") == "UA") \
                  .filter(col("origin") == "SEA") \
                  .agg(f.avg(f.col('air_time')).alias('avg_airtime (mins)')) \
                  .withColumn("avg_airtime (mins)", f.round(col('avg_airtime (mins)'), 2))
average_duration.show()

# #Find the total flight duration in hours.
total_airtime = flights_data_final.filter(col("carrier") == "UA") \
                  .filter(col("origin") == "SEA") \
                  .agg(f.sum(f.col('air_time')).alias('total_airtime (hrs)')) \
                  .withColumn("total_airtime (hrs)", f.round(col('total_airtime (hrs)') / 60, 2))
total_airtime.show()

+------------------+
|avg_airtime (mins)|
+------------------+
|            181.64|
+------------------+

+-------------------+
|total_airtime (hrs)|
+-------------------+
|            2107.02|
+-------------------+



## 3e)

In [24]:
#Read planes_csv and store in dataframe. Show rows, schema and number of occurences.
planes_data = spark\
.read\
.option("inferSchema", "true")\
.option("header", "true")\
.csv ("planes_data_v2.csv")

In [25]:
print(type(planes_data))

<class 'pyspark.sql.dataframe.DataFrame'>


In [26]:
planes_data.show()

+-------+----+--------------------+----------------+--------+-------+-----+-----+---------+
|tailnum|year|                type|    manufacturer|   model|engines|seats|speed|   engine|
+-------+----+--------------------+----------------+--------+-------+-----+-----+---------+
| N102UW|1998|Fixed wing multi ...|AIRBUS INDUSTRIE|A320-214|      2|  182| null|Turbo-fan|
| N103US|1999|Fixed wing multi ...|AIRBUS INDUSTRIE|A320-214|      2|  182| null|Turbo-fan|
| N104UW|1999|Fixed wing multi ...|AIRBUS INDUSTRIE|A320-214|      2|  182| null|Turbo-fan|
| N105UW|1999|Fixed wing multi ...|AIRBUS INDUSTRIE|A320-214|      2|  182| null|Turbo-fan|
| N107US|1999|Fixed wing multi ...|AIRBUS INDUSTRIE|A320-214|      2|  182| null|Turbo-fan|
| N108UW|1999|Fixed wing multi ...|AIRBUS INDUSTRIE|A320-214|      2|  182| null|Turbo-fan|
| N109UW|1999|Fixed wing multi ...|AIRBUS INDUSTRIE|A320-214|      2|  182| null|Turbo-fan|
| N110UW|1999|Fixed wing multi ...|AIRBUS INDUSTRIE|A320-214|      2|  182| null

In [27]:
planes_data.count()

2628

In [28]:
planes_data.printSchema()

root
 |-- tailnum: string (nullable = true)
 |-- year: integer (nullable = true)
 |-- type: string (nullable = true)
 |-- manufacturer: string (nullable = true)
 |-- model: string (nullable = true)
 |-- engines: integer (nullable = true)
 |-- seats: integer (nullable = true)
 |-- speed: integer (nullable = true)
 |-- engine: string (nullable = true)



In [29]:
#Delete the “speed” column and change the name of “year” column to “plane_year”
planes_data_new = planes_data.withColumnRenamed("year", "plane_year").drop("speed")
planes_data_new.show()

+-------+----------+--------------------+----------------+--------+-------+-----+---------+
|tailnum|plane_year|                type|    manufacturer|   model|engines|seats|   engine|
+-------+----------+--------------------+----------------+--------+-------+-----+---------+
| N102UW|      1998|Fixed wing multi ...|AIRBUS INDUSTRIE|A320-214|      2|  182|Turbo-fan|
| N103US|      1999|Fixed wing multi ...|AIRBUS INDUSTRIE|A320-214|      2|  182|Turbo-fan|
| N104UW|      1999|Fixed wing multi ...|AIRBUS INDUSTRIE|A320-214|      2|  182|Turbo-fan|
| N105UW|      1999|Fixed wing multi ...|AIRBUS INDUSTRIE|A320-214|      2|  182|Turbo-fan|
| N107US|      1999|Fixed wing multi ...|AIRBUS INDUSTRIE|A320-214|      2|  182|Turbo-fan|
| N108UW|      1999|Fixed wing multi ...|AIRBUS INDUSTRIE|A320-214|      2|  182|Turbo-fan|
| N109UW|      1999|Fixed wing multi ...|AIRBUS INDUSTRIE|A320-214|      2|  182|Turbo-fan|
| N110UW|      1999|Fixed wing multi ...|AIRBUS INDUSTRIE|A320-214|      2|  182

In [30]:
#inner join with the existing flight DataFrame based on the key of tailnum
flights_plane = planes_data_new.join(flights_data_final,
               on='tailnum',how="inner")
flights_plane.show()

+-------+----------+--------------------+--------------+-----------+-------+-----+---------+----+-----+---+--------+---------+--------+---------+-------+------+------+----+--------+--------+----+------+----------------------------+
|tailnum|plane_year|                type|  manufacturer|      model|engines|seats|   engine|year|month|day|dep_time|dep_delay|arr_time|arr_delay|carrier|flight|origin|dest|air_time|distance|hour|minute|flight_speed(miles per hour)|
+-------+----------+--------------------+--------------+-----------+-------+-----+---------+----+-----+---+--------+---------+--------+---------+-------+------+------+----+--------+--------+----+------+----------------------------+
| N846VA|      2011|Fixed wing multi ...|        AIRBUS|   A320-214|      2|  182|Turbo-fan|2014|   12|  8|     658|       -7|     935|       -5|     VX|  1780|   SEA| LAX|     132|     954|   6|    58|                      433.64|
| N559AS|      2006|Fixed wing multi ...|        BOEING|    737-890|    

In [31]:
#the count after inner join
flights_plane.count()

9394

In [32]:
#Display rows with missing values 
flight_plane_missing = flights_plane.where(reduce(lambda x, y: x | y, (f.col(x).isNull() for x in flights_plane.columns)))
flight_plane_missing.show()

+-------+----------+--------------------+----------------+-------------+-------+-----+-------------+----+-----+---+--------+---------+--------+---------+-------+------+------+----+--------+--------+----+------+----------------------------+
|tailnum|plane_year|                type|    manufacturer|        model|engines|seats|       engine|year|month|day|dep_time|dep_delay|arr_time|arr_delay|carrier|flight|origin|dest|air_time|distance|hour|minute|flight_speed(miles per hour)|
+-------+----------+--------------------+----------------+-------------+-------+-----+-------------+----+-----+---+--------+---------+--------+---------+-------+------+------+----+--------+--------+----+------+----------------------------+
| N918SW|      null|Fixed wing multi ...|  BOMBARDIER INC|  CL-600-2B19|      2|   55|    Turbo-fan|2014|   12| 11|    1057|       87|    1315|      112|     OO|  6328|   PDX| SFO|     118|     550|  10|    57|                      279.66|
| N527AS|      null|Fixed wing multi ...

In [33]:
#Display the count of missing values
flight_plane_missing.count()

91

In [34]:
#Drop missing values 
flights_plane_new = flights_plane.na.drop()

#Show the count of rows in the original and cleaned DataFrames
print("Original DataFrame Count:", flights_plane.count())
print("Cleaned DataFrame Count:", flights_plane_new.count())

Original DataFrame Count: 9394
Cleaned DataFrame Count: 9303


In [35]:
flights_plane_new.show()

+-------+----------+--------------------+--------------+-----------+-------+-----+---------+----+-----+---+--------+---------+--------+---------+-------+------+------+----+--------+--------+----+------+----------------------------+
|tailnum|plane_year|                type|  manufacturer|      model|engines|seats|   engine|year|month|day|dep_time|dep_delay|arr_time|arr_delay|carrier|flight|origin|dest|air_time|distance|hour|minute|flight_speed(miles per hour)|
+-------+----------+--------------------+--------------+-----------+-------+-----+---------+----+-----+---+--------+---------+--------+---------+-------+------+------+----+--------+--------+----+------+----------------------------+
| N846VA|      2011|Fixed wing multi ...|        AIRBUS|   A320-214|      2|  182|Turbo-fan|2014|   12|  8|     658|       -7|     935|       -5|     VX|  1780|   SEA| LAX|     132|     954|   6|    58|                      433.64|
| N559AS|      2006|Fixed wing multi ...|        BOEING|    737-890|    

In [36]:
#Find the Top 20 planes (e.g., carrier, model, plane_year) that made
#the most number of trips. Sort the results from highest to lowest counts.
flights_plane_new.groupBy("carrier","model","plane_year") \
  .count() \
  .sort(desc("count")) \
  .limit(20) \
  .show()


#Find the Bottom 20 planes (e.g., carrier, model, plane_year) planes (e.g., carrier, model, plane_year) that
#made the least number of trips. Sort the results from the lowest to highest counts.
flights_plane_new.groupBy("carrier","model","plane_year") \
  .count() \
  .sort(asc("count")) \
  .limit(20) \
  .show()

+-------+-----------+----------+-----+
|carrier|      model|plane_year|count|
+-------+-----------+----------+-----+
|     AS|    737-890|      2007|  343|
|     AS|  737-990ER|      2013|  330|
|     AS|    737-890|      2008|  308|
|     OO|CL-600-2C10|      2001|  300|
|     AS|    737-490|      1998|  282|
|     AS|    737-890|      2006|  281|
|     AS|    737-790|      1999|  256|
|     AS|    737-890|      2009|  209|
|     AS|  737-990ER|      2014|  206|
|     AS|    737-790|      2000|  161|
|     AS|    737-990|      2003|  142|
|     AS|    737-990|      2001|  133|
|     AS|  737-990ER|      2012|  128|
|     AS|    737-790|      2001|  120|
|     OO|CL-600-2D24|      2006|  118|
|     AS|    737-890|      2010|  109|
|     DL|    737-832|      2001|  108|
|     AS|    737-4Q8|      1992|   93|
|     WN|    737-7H4|      2004|   90|
|     AS|    737-490|      1997|   89|
+-------+-----------+----------+-----+

+-------+--------------+----------+-----+
|carrier|         mod

In [37]:
#SQL Find the Top 20 planes (e.g., carrier, model, plane_year) that made
#the most number of trips. Sort the results from highest to lowest counts.
flights_plane_new.createOrReplaceTempView("flights_plane_new")
spark.sql("""
SELECT carrier, model, plane_year, COUNT(*) as count
FROM flights_plane_new
GROUP BY carrier, model, plane_year
ORDER BY count DESC
LIMIT 20
""").show()



#SQL Find the Bottom 20 planes (e.g., carrier, model, plane_year) planes (e.g., carrier, model, plane_year) that
#made the least number of trips. Sort the results from the lowest to highest counts.
spark.sql("""
SELECT carrier, model, plane_year, COUNT(*) as count
FROM flights_plane_new
GROUP BY carrier, model, plane_year
ORDER BY count ASC
LIMIT 20
""").show()

+-------+-----------+----------+-----+
|carrier|      model|plane_year|count|
+-------+-----------+----------+-----+
|     AS|    737-890|      2007|  343|
|     AS|  737-990ER|      2013|  330|
|     AS|    737-890|      2008|  308|
|     OO|CL-600-2C10|      2001|  300|
|     AS|    737-490|      1998|  282|
|     AS|    737-890|      2006|  281|
|     AS|    737-790|      1999|  256|
|     AS|    737-890|      2009|  209|
|     AS|  737-990ER|      2014|  206|
|     AS|    737-790|      2000|  161|
|     AS|    737-990|      2003|  142|
|     AS|    737-990|      2001|  133|
|     AS|  737-990ER|      2012|  128|
|     AS|    737-790|      2001|  120|
|     OO|CL-600-2D24|      2006|  118|
|     AS|    737-890|      2010|  109|
|     DL|    737-832|      2001|  108|
|     AS|    737-4Q8|      1992|   93|
|     WN|    737-7H4|      2004|   90|
|     AS|    737-490|      1997|   89|
+-------+-----------+----------+-----+

+-------+--------------+----------+-----+
|carrier|         mod

In [38]:
spark.stop()